In [1]:
%cd ..
%pwd

/root/CoTFaithChecker


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/root/CoTFaithChecker'

In [2]:
import os
import logging
import json

from a_confirm_posthoc.src.main.pipeline import load_model_and_tokenizer, generate_dataset_completions
from a_confirm_posthoc.src.eval.llm_verificator import run_verification
from a_confirm_posthoc.src.eval.switch_check import run_switch_check
from a_confirm_posthoc.src.eval.llm_hint_verificator import run_hint_verification

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
#model_path = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
model, tokenizer, model_name, device = load_model_and_tokenizer(model_path)

2025-04-23 12:31:26,427 - INFO - CUDA is available. Using GPU.
2025-04-23 12:31:26,427 - INFO - Loading model and tokenizer: deepseek-ai/DeepSeek-R1-Distill-Qwen-14B onto cuda
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.95it/s]
2025-04-23 12:31:34,955 - INFO - Model and tokenizer loaded successfully.


In [4]:
dataset_name = "mmlu"
hint_types = ["unethical_information", "induced_urgency"]
n_questions = 500

In [ ]:
generate_dataset_completions(
    model = model,
    tokenizer = tokenizer,
    model_name = model_name,
    device = device,
    dataset_name = dataset_name,
    hint_types = hint_types,
    batch_size = 16,
    max_new_tokens = None, 
    n_questions = n_questions
)

2025-04-23 12:31:34,973 - INFO - Using chat template: User: {instruction}
Assistant:
2025-04-23 12:31:34,973 - INFO - --- Processing dataset for hint type: unethical_information ---
2025-04-23 12:31:34,981 - INFO - Generating completions for unethical_information...
2025-04-23 12:31:34,982 - INFO - Using max_new_tokens: 2048
2025-04-23 12:31:34,982 - INFO - Processing batch 1/16 (Size: 32, QIDs: 0-31)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warni

OutOfMemoryError: CUDA out of memory. Tried to allocate 446.00 MiB. GPU 0 has a total capacity of 39.38 GiB of which 439.88 MiB is free. Process 3088185 has 38.93 GiB memory in use. Of the allocated memory 36.44 GiB is allocated by PyTorch, and 2.00 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Run llm verification to get the final model answers
# Note that this will drop the results that are N/A (eg the model never stopped reasoning)
run_verification(dataset_name, hint_types, model_name, n_questions)

In [ ]:
# Check if the model switches between none and the other hint types
# [1:] because we don't want to check the none hint type as it's the baseline
run_switch_check(dataset_name, hint_types[1:], model_name, n_questions)

In [ ]:
# Verify if the model verbalizes the hint
# [1:] because we don't want to check the none hint type as it's the baseline
run_hint_verification(dataset_name, hint_types[1:], model_name, n_questions)

In [ ]:
from a_confirm_posthoc.src.eval.faithfulness_metric import run_faithfulness_metric
base_path = "data/mmlu/DeepSeek-R1-Distill-Llama-8B/induced_urgency/"

hint_verification_path = base_path + "hint_verification_with_500.json"
switch_analysis_path = base_path + "switch_analysis_with_500.json"

results = run_faithfulness_metric(
    hint_verification_path=hint_verification_path,
    switch_analysis_path=switch_analysis_path,
    out_filename=base_path+"/faithfulness_results.json"
)

print("=== Faithfulness Results ===")
print("Raw faithfulness:       ", results["raw_faithfulness"])
print("Corrected faithfulness: ", results["corrected_faithfulness"])
print("Alpha:                  ", results["alpha"])
print("p (switch-to-hint):     ", results["p"])
print("q (switch-other):       ", results["q"])

In [ ]:
from a_confirm_posthoc.src.eval.faithfulness_metric import compute_faithfulness_metric

unhinted_path = "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/none/completions_with_150.json"

In [ ]:
# Sycophancy

verification_path = "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/sycophancy/hint_verification_with_150.json"
hinted_path   = "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/sycophancy/completions_with_150.json"
hints_path    = "data/gsm8k/hints_sycophancy.json"


results = compute_faithfulness_metric(
    unhinted_completions_path=unhinted_path,
    hinted_completions_path=hinted_path,
    hint_verification_path=verification_path,
    hints_path=hints_path,
)

print("Faithfulness Metric Results:")
for k, v in results.items():
    print(f"  {k}: {v}")

In [ ]:
# Induced Urgency

verification_path = "data/induced_urgency/hint_verification_DeepSeek-R1-Distill-Llama-8B_with_150.json"
hinted_path   = "data/induced_urgency/completions_DeepSeek-R1-Distill-Llama-8B_with_150.json"
hints_path    = "data/induced_urgency/hints.json"

results = compute_faithfulness_metric(
    unhinted_completions_path=unhinted_path,
    hinted_completions_path=hinted_path,
    hint_verification_path=verification_path,
    hints_path=hints_path,
)

print("Faithfulness Metric Results:")
for k, v in results.items():
    print(f"  {k}: {v}")

In [ ]:
# Unethical Info

verification_path = "data/unethical_information/hint_verification_DeepSeek-R1-Distill-Llama-8B_with_150.json"
hinted_path   = "data/unethical_information/completions_DeepSeek-R1-Distill-Llama-8B_with_150.json"
hints_path    = "data/unethical_information/hints.json"

results = compute_faithfulness_metric(
    unhinted_completions_path=unhinted_path,
    hinted_completions_path=hinted_path,
    hint_verification_path=verification_path,
    hints_path=hints_path,
)

print("Faithfulness Metric Results:")
for k, v in results.items():
    print(f"  {k}: {v}")